Clientes recorrentes.

In [ ]:
import pandas as pd

df_orders = pd.read_csv('Order_semicolon.csv', sep=';')

customer_orders = df_orders['customer'].value_counts()

new_customers = customer_orders[customer_orders == 1].index
returning_customers = customer_orders[customer_orders > 1].index

total_customers = len(customer_orders)
num_new_customers = len(new_customers)
percentage_new = (num_new_customers / total_customers) * 100

print("### Análise de Novos Clientes ###")
print(f"Total de clientes: {total_customers}")
print(f"Clientes novos com 1 pedido: {num_new_customers}")
print(f"Clientes recorrentes com 1 ou mais pedidos: {len(returning_customers)}")
print(f"Porcentagem de clientes novos: {percentage_new:.2f}%\n")

multi_order_customers = customer_orders[customer_orders >= 3].index

if not multi_order_customers.empty:
    print("### Análise de Clientes com 3+ Pedidos ###")

    top_customers_info = []

    for customer_id in multi_order_customers:

      customer_df = df_orders[df_orders['customer'] == customer_id]
      most_common_channel = customer_df['salesChannel'].mode()[0]

      top_customers_info.append({
            'Cliente': customer_id,
            'Total de Pedidos': len(customer_df),
            'Canal de Venda Mais Usado': most_common_channel
        })

      top_customers_df = pd.DataFrame(top_customers_info).sort_values(
        by='Total de Pedidos', ascending=False
    ).reset_index(drop=True)

    print(top_customers_df)

else:
    print("Não foram encontrados clientes com 3 ou mais pedidos.")

10 Campanhas com mais pedidos.

In [25]:
import pandas as pd

df_pedidos = pd.read_csv('Order_semicolon.csv', sep=';')
df_campanhas = pd.read_csv('CampaignQueue_semicolon.csv', sep=';')

pedidos_por_cliente = df_pedidos['customer'].value_counts().reset_index()
pedidos_por_cliente.columns = ['customer_id', 'total_pedidos']

df_analise = pd.merge(df_campanhas, pedidos_por_cliente, left_on='customerId', right_on='customer_id', how='left')

df_analise['total_pedidos'] = df_analise['total_pedidos'].fillna(0).astype(int)

total_pedidos_por_campanha = df_analise.groupby('campaignId')['total_pedidos'].sum().sort_values(ascending=False).reset_index()

print("Quantidade Total de Pedidos por Campanha")
print("-" * 60)
print(total_pedidos_por_campanha.head(10).to_string(index=False))

Quantidade Total de Pedidos por Campanha
------------------------------------------------------------
 campaignId  total_pedidos
        573             27
         98             20
        692             20
        123             19
       1708             19
       1117             19
       1945             18
       1124             18
       1841             18
        296             18


10 Campanhas maior média de pedidos.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df_pedidos = pd.read_csv('Order_semicolon.csv', sep=';')
df_campanhas = pd.read_csv('Campaign_semicolon.csv', sep=';')
df_fila_campanhas = pd.read_csv('CampaignQueue_semicolon.csv', sep=';')

pedidos_por_cliente = df_pedidos['customer'].value_counts().reset_index()
pedidos_por_cliente.columns = ['customerId', 'total_pedidos']

df_analise = pd.merge(df_fila_campanhas, pedidos_por_cliente, on='customerId', how='left')

df_analise['total_pedidos'] = df_analise['total_pedidos'].fillna(0)

media_por_campanha = df_analise.groupby('campaignId')['total_pedidos'].mean().reset_index()
media_por_campanha.columns = ['campaignId', 'media_pedidos']

df_nomes_campanhas = df_campanhas[['id', 'name']]
df_resultado = pd.merge(media_por_campanha, df_nomes_campanhas, left_on='campaignId', right_on='id', how='left')

df_resultado = df_resultado.sort_values(by='media_pedidos', ascending=False).head(10)

plt.figure(figsize=(12, 8))
plt.barh(df_resultado['name'], df_resultado['media_pedidos'], color='skyblue')
plt.title('Top 10 Campanhas com Maior Média de Pedidos por Cliente', fontsize=16)
plt.xlabel('Média de Pedidos por Cliente', fontsize=12)
plt.ylabel('Nome da Campanha', fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()